# DMRG algorithm

This tutorial demonstrates an application of the DMRG algorithm to find the ground state of the Fermi-Hubbard model on a one-dimensional lattice.

In [1]:
# import NumPy and the PyTeNet package
import numpy as np
import pytenet as ptn

## Construct Fermi-Hubbard Hamiltonian

In [2]:
# number of spin-endowed lattice sites (local dimension is 4)
nsites = 6

# Hamiltonian parameters
t  = 1.0
u  = 4.0
mu = 1.5

In [3]:
# construct the Fermi-Hubbard Hamiltonian as matrix product operator (MPO)
hamiltonian = ptn.fermi_hubbard_mpo(nsites, t, u, mu)

In [4]:
# local physical quantum numbers (particle number and spin)
[ptn.decode_quantum_number_pair(qnum) for qnum in hamiltonian.qd]

[(0, 0), (1, -1), (1, 1), (2, 0)]

In [5]:
# virtual bond dimensions
hamiltonian.bond_dims

[1, 6, 6, 6, 6, 6, 1]

## Run two-site DMRG

In [6]:
# overall quantum number sector of quantum state (particle number and spin)
q_pnum = 7
q_spin = 1
qnum_sector = ptn.encode_quantum_number_pair(q_pnum, q_spin)

In [7]:
# random initial MPS
rng = np.random.default_rng(42)
psi = ptn.MPS.construct_random(nsites, hamiltonian.qd, qnum_sector, max_vdim=18, dtype='real', rng=rng)

# actually run DMRG; 'psi' is updated in-place
numsweeps = 4
en_sweeps = ptn.dmrg_twosite(hamiltonian, psi, numsweeps, tol_split=1e-8)

# value after last iteration
e0 = en_sweeps[-1]
e0

c:\code\pytenet\pytenet\krylov.py:41: RuntimeWarning: beta[0] ~= 0 encountered during Lanczos iteration.
  warnings.warn(


-18.484358904033297

In [8]:
psi.bond_dims

[1, 4, 16, 30, 16, 4, 1]

## Evaluate results and compare with exact diagonalization

In [9]:
# represent 'psi' as vector
psi_vec = psi.as_vector()
psi_vec.shape

(4096,)

In [10]:
# must be normalized
np.linalg.norm(psi_vec)

1.0000000000000002

In [11]:
# energy after each DMRG sweep
en_sweeps

array([-18.4843589, -18.4843589, -18.4843589, -18.4843589])

In [12]:
# construct the (dense) matrix representation of the matrix product operator on the full Hilbert space
h_mat = hamiltonian.as_matrix()
# Hilbert space dimension is 4^nsites
h_mat.shape

(4096, 4096)

In [13]:
# check consistency with energy expectation value (difference should be numerically zero):
abs(np.vdot(psi_vec, h_mat @ psi_vec) - e0)

1.4210854715202004e-14

In [14]:
# reference eigenvalues (based on exact diagonalization of matrix representation)
w_ref = np.linalg.eigvalsh(h_mat)
# reference ground state energy
w_ref[0]

-18.484358962762247

In [15]:
# difference to reference ground state energy
en_sweeps - w_ref[0]

array([5.87289257e-08, 5.87289755e-08, 5.87289577e-08, 5.87289506e-08])